In [1]:
import sys
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/database_functions/')
from database import Database
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/modules/')
from utils import Format_data
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

db = Database()
fd = Format_data()

2025-10-17 14:19:03,483 - INFO - Cliente Oracle inicializado com sucesso usando o caminho: H:\Tecnologia\EQUIPE - DADOS\dev_env\000_base\instantclient_19_21
2025-10-17 14:19:07,161 - INFO - Banco de dados conectado com sucesso!


In [2]:
# BLOCO 1 — Coleta de dados
from datetime import datetime
import pandas as pd

hoje = datetime.today().date()
ontem = hoje - pd.Timedelta(days=1)
mes_referencia = hoje.strftime('%m/%Y')


# , 'CAMP_OFTALMOLOGIA', 'CAMP_PSICO_PSIQUI', 'CAMP_ODONTO','CAMP_DERMATOLOGIA', 'CAMP_CHECKUP_MULHER', 'CAMP_CHECKUP_SAUDE', 'CAMP_ODONTO_GERAL'

tabelas_campanhas = [
    'CAMP_CARDIOLOGIA','CAMP_OFTALMOLOGIA', 'CAMP_PSICO_PSIQUI', 'CAMP_ODONTO','CAMP_DERMATOLOGIA', 'CAMP_CHECKUP_MULHER', 'CAMP_CHECKUP_SAUDE', 'CAMP_ODONTO_GERAL'

]

registros_por_campanha = {}

for tabela in tabelas_campanhas:
    # print(f"\n🔍 Campanha: {tabela}")

    # 1. Total do mailing (sem filtro de data) -- esse qui quero trazer apenas desse mês para ver a quantidade total de meiling de cada campanha.
    query_total = f"""
              SELECT *
        FROM {tabela}
        WHERE TO_CHAR(LIGAR_EM, 'MM/YYYY') = '{mes_referencia}'
        
    """
    df_total = pd.read_sql(query_total, con=db.connection)

    # 2. Registros entre 01/mês e ontem -- Ja esse quero ver todos status dentro do mês.
    query_periodo = f"""
        SELECT * 
        FROM {tabela}
        WHERE TO_CHAR(LIGAR_EM, 'MM/YYYY') = '{mes_referencia}'
        AND LIGAR_EM BETWEEN TO_DATE('01/{hoje.strftime('%m/%Y')}', 'DD/MM/YYYY') AND TO_DATE('{ontem.strftime('%Y-%m-%d')}', 'YYYY-MM-DD')
    """
    df_periodo = pd.read_sql(query_periodo, con=db.connection)

    registros_por_campanha[tabela] = {
        'total': df_total.copy(),
        'periodo': df_periodo.copy()
    }



C:\Users\luci.abreu\AppData\Local\Temp\ipykernel_12376\3167845054.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_total = pd.read_sql(query_total, con=db.connection)
C:\Users\luci.abreu\AppData\Local\Temp\ipykernel_12376\3167845054.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_periodo = pd.read_sql(query_periodo, con=db.connection)


In [3]:
# BLOCO 2 — Calculo de métricas e visualizações

import pandas as pd

resumo = []
ligacoes_por_dia = []
tempo_medio_por_campanha = []
distribuicao_por_canal = []
tentativas_por_paciente = []

for tabela, dados in registros_por_campanha.items():
    df_total = dados['total']
    df_periodo = dados['periodo']

    # Nome real da campanha
    nome_real = df_total['CAMPANHA'].mode()[0] if not df_total['CAMPANHA'].isna().all() else tabela

    # Métricas básicas
    total_mailing = len(df_total)
    df_realizadas = df_periodo[df_periodo['TEMPO_LIGACAO'] > 0]
    realizadas = len(df_realizadas)
    taxa_conversao = round((realizadas / total_mailing) * 100, 2) if total_mailing > 0 else 0
    tempo_medio = round(df_realizadas['TEMPO_LIGACAO'].mean(), 2) if realizadas > 0 else 0
    status_contagem = df_periodo['STATUS'].value_counts().to_dict()
    canal_contagem = df_periodo['CANAIS'].value_counts().to_dict()

    # Métricas de oportunidade
    df_nao_contatados = df_periodo[df_periodo['TEMPO_LIGACAO'] == 0]
    nao_contatados = len(df_nao_contatados)
    taxa_nao_contato = round((nao_contatados / total_mailing) * 100, 2) if total_mailing > 0 else 0

    df_pendentes = df_periodo[df_periodo['STATUS'].isin(['Enviar', 'Em Execucao'])]
    pendentes = len(df_pendentes)
    taxa_pendentes = round((pendentes / total_mailing) * 100, 2) if total_mailing > 0 else 0

    # Pacientes com múltiplas tentativas (mesmo COD_PACIENTE aparece mais de uma vez)
    pacientes_multiplos = df_periodo['COD_PACIENTE'].value_counts()
    tentativas_multiplas = pacientes_multiplos[pacientes_multiplos > 1].count()

    # Tentativas por paciente (para análise futura)
    tentativas_por_paciente.append(
        pacientes_multiplos.rename(nome_real)
    )

    resumo.append({
    'CAMPANHA': nome_real,
    'TOTAL MAILING': total_mailing,
    'LIGACOES REALIZADAS': realizadas,
    'TAXA CONVERSAO %': taxa_conversao,
    'TEMPO MEDIO LIGACAO': tempo_medio,
    'NAO CONTATADOS': nao_contatados,
    'TAXA NAO CONTATO %': taxa_nao_contato,
    'PENDENTES': pendentes,
    'TAXA PENDENTES %': taxa_pendentes,
    'TENTATIVAS MULTIPLAS': tentativas_multiplas,
    **status_contagem
})

    # Ligações por dia
    ligacoes_por_dia.append(
        df_realizadas.groupby('LIGAR_EM').size().rename(nome_real)
    )

    # Tempo médio por campanha
    tempo_medio_por_campanha.append((nome_real, tempo_medio))

    # Distribuição por canal
    distribuicao_por_canal.append(
        pd.DataFrame.from_dict(canal_contagem, orient='index', columns=[nome_real])
    )

# 🔹 DataFrame resumo
df_resumo = pd.DataFrame(resumo).fillna(0)

# 🔹 DataFrame de tentativas por paciente (opcional para análise futura)
df_tentativas = pd.concat(tentativas_por_paciente, axis=1).fillna(0).astype(int)




In [4]:
# BLOCO 3 — Visualizações e relatório HTML
import pandas as pd
import plotly.express as px
import plotly.io as pio

# 🔹 Gráfico 1: Ligações realizadas
fig1 = px.bar(df_resumo, x='CAMPANHA', y='LIGACOES REALIZADAS', text='LIGACOES REALIZADAS',
              title='Ligações Realizadas até ontem', color='CAMPANHA')
fig1.update_traces(textposition='outside')
fig1.update_layout(xaxis_title='Campanha', yaxis_title='Quantidade')

# 🔹 Gráfico 2: Taxa de conversão
fig2 = px.bar(df_resumo, x='CAMPANHA', y='TAXA CONVERSAO %', text='TAXA CONVERSAO %',
              title='Taxa de Conversão (%) por Campanha', color='CAMPANHA')
fig2.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig2.update_layout(xaxis_title='Campanha', yaxis_title='Taxa de Conversão (%)')

# 🔹 Gráfico 3: Tempo médio de ligação
fig3 = px.bar(df_resumo, x='CAMPANHA', y='TEMPO MEDIO LIGACAO', text='TEMPO MEDIO LIGACAO',
              title='Tempo Médio de Ligação por Campanha', color='CAMPANHA')
fig3.update_traces(texttemplate='%{text:.1f}', textposition='outside')
fig3.update_layout(xaxis_title='Campanha', yaxis_title='Tempo Médio (segundos)')

# 🔹 Gráfico 4: Taxa de não contato
fig4 = px.bar(df_resumo, x='CAMPANHA', y='TAXA NAO CONTATO %', text='TAXA NAO CONTATO %',
              title='Taxa de Não Contato (%) por Campanha', color='CAMPANHA')
fig4.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig4.update_layout(xaxis_title='Campanha', yaxis_title='Não Contatados (%)')

# 🔹 Gráfico 5: Taxa de pendentes
fig5 = px.bar(df_resumo, x='CAMPANHA', y='TAXA PENDENTES %', text='TAXA PENDENTES %',
              title='Taxa de Pendências (%) por Campanha', color='CAMPANHA')
fig5.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig5.update_layout(xaxis_title='Campanha', yaxis_title='Pendentes (%)')

# 🔹 Gráfico 6: Tentativas múltiplas por campanha
fig6 = px.bar(df_resumo, x='CAMPANHA', y='TENTATIVAS MULTIPLAS', text='TENTATIVAS MULTIPLAS',
              title='Pacientes com Múltiplas Tentativas por Campanha', color='CAMPANHA')
fig6.update_traces(textposition='outside')
fig6.update_layout(xaxis_title='Campanha', yaxis_title='Quantidade')

# 🔹 Tabela resumo como HTML
tabela_html = df_resumo.to_html(index=False, classes='table table-bordered', border=0)

# 🔹 Monta o HTML final
html_final = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Relatório Estratégico de Campanhas</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 40px;
        }}
        h1 {{
            text-align: center;
            color: #333;
        }}
        .table {{
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 40px;
        }}
        .table th, .table td {{
            border: 1px solid #ccc;
            padding: 8px;
            text-align: center;
            font-size: 13px; /* ou 12px, 11px, etc. */
        }}
        .grafico {{
            margin-bottom: 60px;
        }}
    </style>
</head>
<body>
    <h1>📊 Relatório Estratégico de Campanhas Telefônicas</h1>
    <h2>Resumo por Campanha</h2>
    {tabela_html}
    <div class="grafico">{pio.to_html(fig1, include_plotlyjs=False, full_html=False)}</div>
    <div class="grafico">{pio.to_html(fig2, include_plotlyjs=False, full_html=False)}</div>
    <div class="grafico">{pio.to_html(fig3, include_plotlyjs=False, full_html=False)}</div>
    <div class="grafico">{pio.to_html(fig4, include_plotlyjs=False, full_html=False)}</div>
    <div class="grafico">{pio.to_html(fig5, include_plotlyjs=False, full_html=False)}</div>
    <div class="grafico">{pio.to_html(fig6, include_plotlyjs=False, full_html=False)}</div>
</body>
</html>
"""

# 🔹 Salva o HTML final
with open('relatorio_campanhas.html', 'w', encoding='utf-8') as f:
    f.write(html_final)

